Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [54]:
from random import choices
from dataclasses import dataclass
from copy import copy
from random import randint, random, choice, uniform
import lab9_lib

fitness = lab9_lib.make_problem(2)

for n in range(10):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")


print(fitness.calls)

In [55]:
POPULATION_SIZE = 60
OFFSPRING_SIZE = 30
TOURNAMENT_SIZE = 10
MUTATION_PROBABILITY = .15

SEQUENCE_LENGTH = 1000

In [56]:
@dataclass
class Individual:
    fitness: float
    genotype: list[bool]

### Selection

In [57]:
class Tournament():
    def __init__(self, size):
        self.size = size

    def select(self, pop):
        pool = [choice(pop) for _ in range(self.size)]
        champion = max(pool, key=lambda i: i.fitness)
        return champion
    
class TournamentEpsilon():
    def __init__(self, size, scheduler):
        self.size = size
        self.scheduler = scheduler
    
    def select(self, pop):
        if random() > self.scheduler.value():
            pool = [choice(pop) for _ in range(self.size)]
            champion = max(pool, key=lambda i: i.fitness)
            return champion
        else:
            return choice(pop)
        
def tweak(ind: Individual):
    obj = copy(ind)
    pos = randint(0, len(obj.genotype) - 1)
    pos2 = randint(0, len(obj.genotype) - 1)


    obj.genotype[pos] = not obj.genotype[pos]
    obj.genotype[pos2] = not obj.genotype[pos2]

    obj.fitness = None

    return obj     

def hill_climbing(ind: Individual, fitness , n_iter = 10):
    s = copy(ind)
    for _ in range(n_iter):
        for _ in range(n_iter):
            r = tweak(s)
            if fitness(r.genotype) > fitness(s.genotype):
                s = r
        best = s
    
    return best

### Mutation

In [58]:
class Scheduler():
    def __init__(self, alpha, decay_rate = 0):
        self.rate = decay_rate
        self.current_value = alpha
    
    def mutation_prob():
        pass

class ConstantScheduler(Scheduler):
    def value(self):
        return self.current_value

class ExponentialDecayScheduler(Scheduler):
    def value(self):
        return_value = self.current_value
        self.current_value = self.current_value * (1 - self.rate)
       
        return return_value
    
class BitFlip():
    def __init__(self, sequence_length, scheduler, num_flips = 1):
        self.sequence_length = sequence_length
        self.scheduler = scheduler
        self.num_flips = num_flips
    
    def mutate(self, ind: Individual) -> Individual:
        offspring = copy(ind)

        if random() < self.scheduler.value():
            
            for i in range(self.num_flips):
                pos = randint(0, self.sequence_length-1)
                offspring.genotype[pos] = not offspring.genotype[pos]
                #offspring.genotype[pos] = choice([True, False])


            offspring.fitness = None
        
        return offspring



### Recombination

In [59]:
class OnePointCrossover():
    def __init__(self, sequence_length):
        self.sequence_length = sequence_length

    def recombine(self,ind1: Individual, ind2: Individual) -> Individual:
        cut_point = randint(0, self.sequence_length-1)
        offspring1 = Individual(fitness=None,
                            genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
        offspring2 = Individual(fitness=None,
                            genotype=ind2.genotype[:cut_point] + ind1.genotype[cut_point:])

        assert len(offspring1.genotype) == self.sequence_length
        assert len(offspring1.genotype) == self.sequence_length

        return offspring1, offspring2

class UniformCrossover():
    def __init__(self, sequence_length, scheduler):
        self.sequence_length = sequence_length
        self.scheduler = scheduler
    
    def recombine(self, ind1: Individual, ind2: Individual) -> Individual:
        rvector1 = [random() for _ in range(self.sequence_length)]
        rvector2 = [random() for _ in range(self.sequence_length)]

        g1 = [p1 if r <  self.scheduler.value() else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector1)]
        g2 = [p1 if r <  self.scheduler.value() else p2 for p1, p2, r in zip(ind1.genotype, ind2.genotype, rvector2)]

        o1 = Individual(fitness = None, genotype=g1)
        o2 = Individual(fitness = None, genotype=g2)

        return o1, o2



In [60]:
class EASimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.offspring_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)
            print(f'generation:{g}_fitness:{self.population[0].fitness}')

            if self.population[0].fitness == 1:
                break

        return record
    
class HybridSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.select_parent = lambda pop : selection.select(pop)
        self.mutate = lambda ind : mutation.mutate(ind)
        self.recombine = lambda ind1, ind2: recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []

        for _ in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size // 2):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                offspring.append(self.mutate(o1))
                offspring.append(self.mutate(o2))

            #Evaluate offsprings
            for i in range(len(offspring)):
                offspring[i] = hill_climbing(offspring[i],fitness=self.fitness_eval)
                offspring[i].fitness = self.fitness_eval(offspring[i].genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

        return record
    

class AdaptiveSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.mutation = mutation
        self.selection = selection
        self.select_parent = lambda pop : self.selection.select(pop)
        self.mutate = lambda ind : self.mutation.mutate(ind)
        self.recombination = recombination
        self.recombine = lambda ind1, ind2: self.recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []
        explorative_control = 0
        explorative_control_higher = 0

        best = 0

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                o1 = self.mutate(o1)
                o2 = self.mutate(o2)

                offspring.append(o1)
                offspring.append(o2)

            #Evaluate offsprings
            for i in offspring:
                if explorative_control_higher >= 5:
                    i = hill_climbing(i)
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if int(1000* self.population[0].fitness) > int(1000 * best):
                best = self.population[0].fitness
                explorative_control = 0
                explorative_control_higher = 0
            else:
                explorative_control += 1
                explorative_control_higher = 0


                if explorative_control >= 2:
                    self.mutation.scheduler.current_value = 1
                    self.selection.scheduler.current_value = 1

                    explorative_control = 0
                    
                    
                
            print(f'generation:{g}_explorecontrol:{explorative_control}_rate{self.selection.scheduler.current_value}_fitness:{self.population[0].fitness}')
            
            if self.population[0].fitness == 1:
                break

        return record
    

class StepSizeSimulation():
    def __init__(self, population_size, offspring_size, fitness_eval,selection, mutation, recombination, num_gen = 100):
        self.population_size = population_size
        self.offspring_size = offspring_size
        self.fitness_eval = fitness_eval
        self.mutation = mutation
        self.selection = selection
        self.select_parent = lambda pop : self.selection.select(pop)
        self.mutate = lambda ind : self.mutation.mutate(ind)
        self.recombination = recombination
        self.recombine = lambda ind1, ind2: self.recombination.recombine(ind1, ind2)
        self.num_gen = num_gen

    def set_inital_pop(self, population):
        self.population = population

    def simulate(self):
        record = []
        explorative_control = 1
        best = 0

        for g in range(self.num_gen):
            offspring = list()

            #Evolution process
            for _ in range(self.population_size):
                p1 = self.select_parent(self.population)
                p2 = self.select_parent(self.population)

                o1, o2= self.recombine(p1, p2)

                for _ in range(min(explorative_control, 10)):
                    o1 = self.mutate(o1)
                    o2 = self.mutate(o2)

                offspring.append(o1)
                offspring.append(o2)

            #Evaluate offsprings
            for i in offspring:
                i.fitness = self.fitness_eval(i.genotype)

            #Add offsprings into the population and choose the best ones for the next generation  
            self.population.extend(offspring)
            self.population.sort(key=lambda i: i.fitness, reverse=True)
            self.population = self.population[:self.population_size]

            record.append(self.population[0].fitness)

            if int(1000* self.population[0].fitness) > int(1000 * best):
                best = self.population[0].fitness
                explorative_control = 1
            else:
                explorative_control += 1

                if explorative_control >= 5:
                    self.mutation.scheduler.current_value = 1
                    self.selection.scheduler.current_value = 1

                    
                    
                
            print(f'generation:{g}_explorecontrol:{explorative_control}_rate{self.selection.scheduler.current_value}_fitness:{self.population[0].fitness}')
            
            if self.population[0].fitness == 1:
                break

        return record

    



### Initialize Population

In [61]:
population = [
    Individual(
        genotype=[choice((True, False)) for _ in range(SEQUENCE_LENGTH)],
        fitness=None,
    )
    for _ in range(POPULATION_SIZE)
]



### Experiment

#### instance 1

In [62]:
fitness = lab9_lib.make_problem(1)

In [63]:
for i in population:
    i.fitness = fitness(i.genotype)

In [64]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.0001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE , scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)

recombination_scheduler = ExponentialDecayScheduler(alpha=0.5, decay_rate=0.000)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH, scheduler=recombination_scheduler)

sim1 = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim1.set_inital_pop(population)

record = sim1.simulate()

generation:0_explorecontrol:0_rate0.9880711199795563_fitness:0.541
generation:1_explorecontrol:0_rate0.9762845381376548_fitness:0.55
generation:2_explorecontrol:0_rate0.9646385570163959_fitness:0.558
generation:3_explorecontrol:0_rate0.9531314994066535_fitness:0.566
generation:4_explorecontrol:0_rate0.9417617081065252_fitness:0.58
generation:5_explorecontrol:0_rate0.9305275456826744_fitness:0.586
generation:6_explorecontrol:0_rate0.9194273942345079_fitness:0.595
generation:7_explorecontrol:0_rate0.9084596551611751_fitness:0.614
generation:8_explorecontrol:0_rate0.897622748931344_fitness:0.632


generation:9_explorecontrol:0_rate0.886915114855721_fitness:0.634
generation:10_explorecontrol:0_rate0.8763352108622888_fitness:0.642
generation:11_explorecontrol:0_rate0.865881513274222_fitness:0.65
generation:12_explorecontrol:0_rate0.8555525165904533_fitness:0.666
generation:13_explorecontrol:0_rate0.8453467332688572_fitness:0.677
generation:14_explorecontrol:0_rate0.8352626935120188_fitness:0.679
generation:15_explorecontrol:0_rate0.825298945055561_fitness:0.691
generation:16_explorecontrol:0_rate0.8154540529589945_fitness:0.702
generation:17_explorecontrol:0_rate0.8057265993990619_fitness:0.715
generation:18_explorecontrol:0_rate0.7961151834655501_fitness:0.722
generation:19_explorecontrol:0_rate0.7866184209595359_fitness:0.732
generation:20_explorecontrol:0_rate0.7772349441940386_fitness:0.745
generation:21_explorecontrol:0_rate0.7679634017970517_fitness:0.749
generation:22_explorecontrol:0_rate0.7588024585169224_fitness:0.757
generation:23_explorecontrol:0_rate0.7497507950300561

In [65]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 26460


#### instance 2

In [66]:
fitness = lab9_lib.make_problem(2)

In [67]:
for i in population:
    i.fitness = fitness(i.genotype)

In [68]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.0001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)

recombination_scheduler = ExponentialDecayScheduler(alpha=0.5, decay_rate=0.000)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH, scheduler=recombination_scheduler)

sim2 = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim2.set_inital_pop(population)

record = sim2.simulate()

generation:0_explorecontrol:0_rate0.9880711199795563_fitness:0.516
generation:1_explorecontrol:1_rate0.9762845381376548_fitness:0.516
generation:2_explorecontrol:0_rate0.9646385570163959_fitness:0.534
generation:3_explorecontrol:1_rate0.9531314994066535_fitness:0.534
generation:4_explorecontrol:0_rate1_fitness:0.534
generation:5_explorecontrol:0_rate0.9880711199795563_fitness:0.54
generation:6_explorecontrol:1_rate0.9762845381376548_fitness:0.54
generation:7_explorecontrol:0_rate1_fitness:0.54
generation:8_explorecontrol:1_rate0.9880711199795563_fitness:0.54
generation:9_explorecontrol:0_rate1_fitness:0.54
generation:10_explorecontrol:1_rate0.9880711199795563_fitness:0.54
generation:11_explorecontrol:0_rate1_fitness:0.54
generation:12_explorecontrol:1_rate0.9880711199795563_fitness:0.54
generation:13_explorecontrol:0_rate1_fitness:0.54
generation:14_explorecontrol:1_rate0.9880711199795563_fitness:0.54
generation:15_explorecontrol:0_rate1_fitness:0.54
generation:16_explorecontrol:1_rate

In [69]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 1.0
num calls: 118500


#### instance 3

In [76]:
fitness = lab9_lib.make_problem(5)

In [77]:
for i in population:
    i.fitness = fitness(i.genotype)

In [78]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE , scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler)
recombination_scheduler = ExponentialDecayScheduler(alpha=0.5, decay_rate=0.000)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH, scheduler=recombination_scheduler)

sim3 = AdaptiveSimulation(population_size=POPULATION_SIZE , offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim3.set_inital_pop(population)

record = sim3.simulate()

generation:0_explorecontrol:0_rate0.8868671875860644_fitness:0.30963
generation:1_explorecontrol:1_rate0.7865334084168147_fitness:0.30963
generation:2_explorecontrol:0_rate0.6975506718651011_fitness:0.31267
generation:3_explorecontrol:0_rate0.6186348025557711_fitness:0.32233999999999996
generation:4_explorecontrol:1_rate0.5486469074854965_fitness:0.32233999999999996
generation:5_explorecontrol:0_rate0.4865769398194536_fitness:0.32527999999999996
generation:6_explorecontrol:1_rate0.43152912216191214_fitness:0.32529
generation:7_explorecontrol:0_rate1_fitness:0.32529
generation:8_explorecontrol:1_rate0.8868671875860644_fitness:0.32539
generation:9_explorecontrol:0_rate1_fitness:0.32549
generation:10_explorecontrol:1_rate0.8868671875860644_fitness:0.3255
generation:11_explorecontrol:0_rate1_fitness:0.32559
generation:12_explorecontrol:1_rate0.8868671875860644_fitness:0.32568
generation:13_explorecontrol:0_rate1_fitness:0.32577999999999996
generation:14_explorecontrol:1_rate0.8868671875860

KeyboardInterrupt: 

In [ ]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 0.4
num calls: 120300


#### instance 4

In [ ]:
fitness = lab9_lib.make_problem(10)

In [ ]:
for i in population:
    i.fitness = fitness(i.genotype)

In [ ]:
selection_scheduler = ExponentialDecayScheduler(alpha = 1, decay_rate = 0.0001)
#selection_scheduler = ConstantScheduler(alpha = 1)
selection = TournamentEpsilon(size = TOURNAMENT_SIZE, scheduler=selection_scheduler)

mutation_scheduler = ExponentialDecayScheduler(alpha=1, decay_rate=0.0001)
#mutation_scheduler = ConstantScheduler(alpha = 0.2)
mutation = BitFlip(sequence_length= SEQUENCE_LENGTH, scheduler=mutation_scheduler, num_flips=10)
recombination_scheduler = ExponentialDecayScheduler(alpha=0.5, decay_rate=0.000)
recombination = UniformCrossover(sequence_length= SEQUENCE_LENGTH, scheduler=recombination_scheduler)

sim4 = AdaptiveSimulation(population_size=POPULATION_SIZE, offspring_size= OFFSPRING_SIZE, fitness_eval=fitness, selection=selection, mutation=mutation, recombination=recombination, num_gen=1000)

sim4.set_inital_pop(population)

record = sim4.simulate()

generation:0_explorecontrol:0_rate0.9880711199795563_fitness:0.25434685
generation:1_explorecontrol:1_rate0.9762845381376548_fitness:0.25434685
generation:2_explorecontrol:0_rate0.9646385570163959_fitness:0.30636939999999996
generation:3_explorecontrol:1_rate0.9531314994066535_fitness:0.30636939999999996
generation:4_explorecontrol:0_rate1_fitness:0.30636939999999996
generation:5_explorecontrol:1_rate0.9880711199795563_fitness:0.30636939999999996
generation:6_explorecontrol:0_rate1_fitness:0.30636939999999996
generation:7_explorecontrol:1_rate0.9880711199795563_fitness:0.30636939999999996
generation:8_explorecontrol:0_rate1_fitness:0.30636939999999996
generation:9_explorecontrol:1_rate0.9880711199795563_fitness:0.30636939999999996
generation:10_explorecontrol:0_rate1_fitness:0.30636939999999996
generation:11_explorecontrol:1_rate0.9880711199795563_fitness:0.30636939999999996
generation:12_explorecontrol:0_rate1_fitness:0.30636939999999996
generation:13_explorecontrol:1_rate0.9880711199

In [ ]:
print(f'fitness value: {record[-1]}')
print(f'num calls: {fitness.calls}')

fitness value: 0.3359508
num calls: 120420


False